In [ ]:
from shapely.geometry import Point
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster, HeatMap, HeatMapWithTime
import ipywidgets as widgets
from IPython.display import HTML, display
from collections import defaultdict

In [ ]:
eq_table = pd.read_csv('../Data/04_deadly_eq_mag_cleaned.csv', index_col=False)

In [ ]:
eq_table.columns

In [ ]:
eq_table['origin (utc)'] = pd.to_datetime(eq_table['origin (utc)'])
eq_table['origin_utc_string'] = eq_table['origin (utc)'].dt.strftime("%Y-%m-%dT%H:%M:%S")

In [ ]:
eq_table['lat'] = pd.to_numeric(eq_table['lat'], errors='coerce')

In [ ]:
eq_table['long'] = pd.to_numeric(eq_table['long'], errors='coerce')

In [ ]:
def lat_long_to_point(row):
    return Point(row['long'], row['lat'])

In [ ]:
eq_table['position'] = eq_table.apply(lat_long_to_point, axis=1)

In [ ]:
eq_table_geo = gpd.GeoDataFrame(eq_table, crs='epsg:4326', geometry=eq_table['position'])

In [ ]:
eq_table_geo.columns

In [ ]:
eq_table_geo['original_magnitude'] = eq_table_geo['original_magnitude'].fillna(0.0)

In [ ]:
world_map = folium.Map(tiles='Stamen Terrain')

In [ ]:
heatmapdata = [
        (row['geometry'].y, row['geometry'].x, row['original_magnitude']) for idx, row 
            in eq_table_geo.iterrows() 
                if row['geometry'] and not pd.isna(row['geometry'].x) and not pd.isna(row['geometry'].y and not pd.isna(row['original_magnitude']))
    ]

In [ ]:
heatmapdata

In [ ]:
hm = HeatMap(data=heatmapdata)

In [ ]:
hm.add_to(world_map)

In [ ]:
world_map

In [ ]:
data_points = []
max_mag = 0
for idx, row in eq_table_geo.sort_values('origin (utc)').iterrows():
    if row['geometry'] and not pd.isna(row['geometry'].x) and not pd.isna(row['geometry'].y):
        data_points.append((row['origin (utc)'].to_pydatetime(), row['geometry'].y, row['geometry'].x, row['original_magnitude']))
        if row['original_magnitude'] > max_mag:
            max_mag = row['original_magnitude']

by_year=defaultdict(list)
for record in data_points:
    by_year[f'{record[0].strftime("%Y")}'].append([record[1],record[2], 1])
    
over_time = []
time_index = []
for key, values in by_year.items():
    over_time.append(values)
    time_index.append(key)

In [ ]:
over_time

In [ ]:
mapheattime = folium.Map(tiles='Stamen Terrain')
hm = HeatMapWithTime(data=over_time,
    index=time_index,
    auto_play=False,
    max_opacity=0.3,
    )
hm.add_to(mapheattime)

In [ ]:
mapheattime